# Final Capstone: Revisiting the Netflix Prize

## Notebook 3: Correlation and Separation of Training and Quiz Sets

In [1]:
import time
start_time = time.perf_counter()
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)

In [2]:
%%time
# import main df and quiz index list
base_path = 'C:/Users/jnpol/Documents/DS/Data Science/UL/'
quindex = pd.read_parquet(base_path + 'quindex.parquet')
quilist = list(quindex.quindex)
net = pd.read_parquet(base_path + 'net2.parquet')
net.info()
net.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100480507 entries, 93047606 to 6426045
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   rating           float64
 3   day_rated        int16  
 4   mov_year         int16  
 5   avg_rate_pm_pd   float32
 6   avg_rate_pc_pd   float32
 7   cust_day_count   int16  
 8   cust_days_since  int16  
 9   mov_days_since   int16  
 10  mov_avg_rating   float32
 11  cust_avg_rating  float32
 12  mov_day_avg      float32
 13  cust_day_avg     float32
 14  avg_rate_yr      float32
 15  avg_rate_cst_yr  float32
 16  bline_approx     float32
 17  all_ratings      int8   
dtypes: float32(9), float64(1), int16(6), int32(1), int8(1)
memory usage: 6.5 GB
Wall time: 4.21 s


,mov_id,cust_id,rating,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,avg_rate_yr,avg_rate_cst_yr,bline_approx,all_ratings
93047606,16469,562200,4.00,1913,2004,19.74,0.02,3,79,286,3.41,3.82,3.51,3.33,3.51,3.52,3.41,4
95827348,16984,639827,1.00,1854,2004,16.95,0.28,4,512,292,3.22,3.38,3.12,2.00,3.51,2.62,2.86,1
56309414,10277,1828455,2.00,1854,1948,5.03,0.08,10,357,1797,3.93,2.88,3.67,2.40,3.83,2.00,3.23,2
69063104,12501,1583579,nan,2137,1984,19.07,0.09,6,501,1543,3.87,4.07,3.95,4.00,3.71,3.25,3.61,5
60852899,11149,458209,3.00,1506,2002,56.17,0.24,9,316,405,3.14,3.26,3.05,2.78,3.50,3.01,3.32,3


## Correlation Matrix

PCA will be done on highly correlated variables in another notebook once the data is separated into features and target, then split.

In [3]:
%%time
net.corr()

Wall time: 1min 5s


,mov_id,cust_id,rating,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,avg_rate_yr,avg_rate_cst_yr,bline_approx,all_ratings
mov_id,1.00,0.00,0.01,0.00,-0.00,0.04,-0.01,-0.00,-0.01,0.04,0.02,0.00,0.02,0.00,0.01,0.01,0.00,0.01
cust_id,0.00,1.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00
rating,0.01,-0.00,1.00,0.09,-0.08,0.10,-0.10,-0.03,-0.03,0.08,0.36,0.40,0.48,0.61,0.09,0.54,0.51,1.00
day_rated,0.00,0.00,0.09,1.00,0.05,0.05,-0.13,0.06,0.09,0.33,0.10,0.19,0.19,0.15,-0.05,0.15,0.12,0.09
mov_year,-0.00,-0.00,-0.08,0.05,1.00,0.27,-0.11,-0.10,0.02,-0.32,-0.22,0.02,-0.17,-0.03,-0.88,-0.15,-0.05,-0.08
avg_rate_pm_pd,0.04,-0.00,0.10,0.05,0.27,1.00,-0.22,-0.07,-0.11,0.05,0.28,0.09,0.21,0.09,-0.26,0.04,0.03,0.10
avg_rate_pc_pd,-0.01,0.00,-0.10,-0.13,-0.11,-0.22,1.00,0.33,0.09,-0.04,-0.10,-0.25,-0.13,-0.17,0.11,-0.19,-0.04,-0.10
cust_day_count,-0.00,0.00,-0.03,0.06,-0.10,-0.07,0.33,1.00,-0.18,0.17,-0.01,-0.05,-0.01,-0.05,0.11,-0.04,-0.01,-0.03
cust_days_since,-0.01,-0.00,-0.03,0.09,0.02,-0.11,0.09,-0.18,1.00,-0.13,-0.03,-0.11,-0.02,-0.05,-0.03,-0.07,0.03,-0.03
mov_days_since,0.04,0.00,0.08,0.33,-0.32,0.05,-0.04,0.17,-0.13,1.00,0.12,0.11,0.16,0.10,0.39,0.16,0.08,0.08


In [4]:
%%time
# create quiz set from primary df by selecting rows with null values
quiz = net[net.isna().any(axis=1)].copy()

# drop the column 'rating' since it is all nans
quiz.drop(['rating'], 1, inplace=True)

# rename the column that contains the ratings to 'rating'
quiz.rename(columns={'all_ratings': 'rating'}, inplace=True)
quiz.info()
quiz.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 69063104 to 69742988
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mov_id           1408395 non-null  int16  
 1   cust_id          1408395 non-null  int32  
 2   day_rated        1408395 non-null  int16  
 3   mov_year         1408395 non-null  int16  
 4   avg_rate_pm_pd   1408395 non-null  float32
 5   avg_rate_pc_pd   1408395 non-null  float32
 6   cust_day_count   1408395 non-null  int16  
 7   cust_days_since  1408395 non-null  int16  
 8   mov_days_since   1408395 non-null  int16  
 9   mov_avg_rating   1408395 non-null  float32
 10  cust_avg_rating  1408395 non-null  float32
 11  mov_day_avg      1408395 non-null  float32
 12  cust_day_avg     1408395 non-null  float32
 13  avg_rate_yr      1408395 non-null  float32
 14  avg_rate_cst_yr  1408395 non-null  float32
 15  bline_approx     1408395 non-null  float32
 16  rating    

,mov_id,cust_id,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,avg_rate_yr,avg_rate_cst_yr,bline_approx,rating
69063104,12501,1583579,2137,1984,19.07,0.09,6,501,1543,3.87,4.07,3.95,4.00,3.71,3.25,3.61,5
10671786,2096,902509,1987,2003,0.67,0.18,2,72,58,2.76,3.64,2.38,3.55,3.51,3.40,3.36,1
6225903,1220,2538285,2060,2004,59.90,0.08,4,1258,470,4.01,3.61,4.00,2.00,3.51,3.40,2.79,5
98655424,17405,143194,2196,2005,26.13,0.01,1,51,319,3.53,2.00,3.46,2.00,3.55,2.00,3.57,2
50506836,9112,2337699,2229,1999,1.91,0.33,1,534,2179,3.14,3.26,3.67,2.50,3.60,2.95,3.49,4


In [5]:
%%time
# split quiz set into features and target
quiz_target = quiz[['rating']].copy()
quiz_features = quiz.drop(['rating'], 1)
quiz_target.info()
display(quiz_target.head())
quiz_features.info()
quiz_features.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 69063104 to 69742988
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   rating  1408395 non-null  int8 
dtypes: int8(1)
memory usage: 12.1 MB


,rating
69063104,5
10671786,1
6225903,5
98655424,2
50506836,4


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408395 entries, 69063104 to 69742988
Data columns (total 16 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mov_id           1408395 non-null  int16  
 1   cust_id          1408395 non-null  int32  
 2   day_rated        1408395 non-null  int16  
 3   mov_year         1408395 non-null  int16  
 4   avg_rate_pm_pd   1408395 non-null  float32
 5   avg_rate_pc_pd   1408395 non-null  float32
 6   cust_day_count   1408395 non-null  int16  
 7   cust_days_since  1408395 non-null  int16  
 8   mov_days_since   1408395 non-null  int16  
 9   mov_avg_rating   1408395 non-null  float32
 10  cust_avg_rating  1408395 non-null  float32
 11  mov_day_avg      1408395 non-null  float32
 12  cust_day_avg     1408395 non-null  float32
 13  avg_rate_yr      1408395 non-null  float32
 14  avg_rate_cst_yr  1408395 non-null  float32
 15  bline_approx     1408395 non-null  float32
dtypes: float32

,mov_id,cust_id,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,avg_rate_yr,avg_rate_cst_yr,bline_approx
69063104,12501,1583579,2137,1984,19.07,0.09,6,501,1543,3.87,4.07,3.95,4.00,3.71,3.25,3.61
10671786,2096,902509,1987,2003,0.67,0.18,2,72,58,2.76,3.64,2.38,3.55,3.51,3.40,3.36
6225903,1220,2538285,2060,2004,59.90,0.08,4,1258,470,4.01,3.61,4.00,2.00,3.51,3.40,2.79
98655424,17405,143194,2196,2005,26.13,0.01,1,51,319,3.53,2.00,3.46,2.00,3.55,2.00,3.57
50506836,9112,2337699,2229,1999,1.91,0.33,1,534,2179,3.14,3.26,3.67,2.50,3.60,2.95,3.49


In [6]:
%%time
# drop the column 'rating' since it is not needed (and float64)
net.drop(['rating'], 1, inplace=True)

# rename the column that contains the ratings to 'rating'
net.rename(columns={'all_ratings': 'rating'}, inplace=True)

# drop the matching rows that belong only in the quiz set
net.drop(quilist, inplace=True)
net.info()
net.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99072112 entries, 93047606 to 6426045
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   day_rated        int16  
 3   mov_year         int16  
 4   avg_rate_pm_pd   float32
 5   avg_rate_pc_pd   float32
 6   cust_day_count   int16  
 7   cust_days_since  int16  
 8   mov_days_since   int16  
 9   mov_avg_rating   float32
 10  cust_avg_rating  float32
 11  mov_day_avg      float32
 12  cust_day_avg     float32
 13  avg_rate_yr      float32
 14  avg_rate_cst_yr  float32
 15  bline_approx     float32
 16  rating           int8   
dtypes: float32(9), int16(6), int32(1), int8(1)
memory usage: 5.6 GB
Wall time: 27.3 s


,mov_id,cust_id,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,avg_rate_yr,avg_rate_cst_yr,bline_approx,rating
93047606,16469,562200,1913,2004,19.74,0.02,3,79,286,3.41,3.82,3.51,3.33,3.51,3.52,3.41,4
95827348,16984,639827,1854,2004,16.95,0.28,4,512,292,3.22,3.38,3.12,2.00,3.51,2.62,2.86,1
56309414,10277,1828455,1854,1948,5.03,0.08,10,357,1797,3.93,2.88,3.67,2.40,3.83,2.00,3.23,2
60852899,11149,458209,1506,2002,56.17,0.24,9,316,405,3.14,3.26,3.05,2.78,3.50,3.01,3.32,3
60355490,11064,1082078,1771,1994,67.86,0.42,24,153,1742,4.09,4.17,4.10,3.96,3.71,3.90,3.50,5


In [7]:
%%time
# split training set into features and target
train_target = net[['rating']].copy()
train_features = net.drop(['rating'], 1)
train_target.info()
print()
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99072112 entries, 93047606 to 6426045
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rating  int8 
dtypes: int8(1)
memory usage: 850.3 MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99072112 entries, 93047606 to 6426045
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   day_rated        int16  
 3   mov_year         int16  
 4   avg_rate_pm_pd   float32
 5   avg_rate_pc_pd   float32
 6   cust_day_count   int16  
 7   cust_days_since  int16  
 8   mov_days_since   int16  
 9   mov_avg_rating   float32
 10  cust_avg_rating  float32
 11  mov_day_avg      float32
 12  cust_day_avg     float32
 13  avg_rate_yr      float32
 14  avg_rate_cst_yr  float32
 15  bline_approx     float32
dtypes: float32(9), int16(6), int32(1)
memory usage: 5.5 GB
Wall time: 3.86 s


In [8]:
%%time
quiz_target.to_parquet('quiz_target.parquet')
del quiz_target
quiz_features.to_parquet('quiz_features.parquet')
del quiz_features
train_target.to_parquet('train_target.parquet')
del train_target
train_features.to_parquet('train_features.parquet')

Wall time: 29.2 s
